In [49]:
import sqlite3
conn=sqlite3.connect('Lab4-131.db')
c=conn.cursor()

In [50]:
c.execute("""CREATE TABLE genes (name TEXT, description TEXT, nucleotide_seq TEXT, length TEXT);""")
c.execute("""CREATE TABLE pathway (name TEXT, enzymes TEXT);""")
c.execute("""CREATE TABLE enzyme(name TEXT, EC TEXT, function TEXT, pathway TEXT,organism TEXT, gene TEXT);""")

In [51]:
from Bio.KEGG import REST
from Bio.KEGG import Enzyme
from Bio.KEGG import Gene
request=REST.kegg_get("ec:3.1.3.10")
open("ec_3.1.3.10.txt",'w').write(request.read())
records=Enzyme.parse(open("ec_3.1.3.10.txt"))
record = list(records)[0]
for x in range(len(record.genes)):
    if record.genes[x][0]=='ECJ':
        eco_gene=record.genes[x][1][0]
eco_gene

'JW0987'

In [52]:
from Bio import SeqIO
from Bio import Entrez
Entrez.email='neilwang@berkeley.edu'
handle=Entrez.esearch(db='nucleotide',term='homo sapiens[ORGN] DLST',sort='relevance',idtype='acc')
for i in Entrez.read(handle)['IdList']:
        handle=Entrez.efetch(db='nucleotide',id=i, rettype='fasta',retmode='text')
        break
for record in SeqIO.parse(handle,'fasta'):
    print(record.seq)

ATGCTGTCCCGGTCCCGCTGTGTGTCTCGGGCGTTCAGCCGCTCGCTCTCCGCCTTCCAGAAGGGGAACTGCCCTCTAGGGAGACGTTCCCTGCCTGGGGTCTCCTTATGCCAGGGACCAGGTTACCCTAACAGCAGGAAGGTTGTCATTAACAACAGTGTCTTCAGTGTTCGCTTTTTCAGAACTACAGCTGTATGCAAGGATGACTTGGTTACAGTCAAAACCCCAGCGTTTGCAGAATCTGTCACAGAGGGAGATGTCAGGTGGGAGAAAGCTGTTGGAGACACAGTTGCAGAAGATGAAGTGGTTTGTGAGATTGAAACTGACAAGACATCTGTGCAGGTTCCATCACCAGCAAATGGCGTGATTGAAGCTCTTTTGGTACCTGATGGGGGAAAAGTCGAAGGAGGCACTCCACTTTTCACACTCAGGAAAACTGGTGCTGCTCCTGCTAAGGCCAAGCCGGCTGAAGCTCCTGCTGCTGCAACCCCAAAAGCAGAACCTACAGCAGCGGCAGTTCCTCCCCCTGCAGCACCCATACCCACTCAGATGCCACCGGTGCCCTCGCCCTCACAACCTCCTTCTGGCAAACCTGTGTCTGCAGTAAAACCCACTGTTGCCCCACCACTAGCTGAGCCAGGAGCTGGCAAAGGTCTGCGTTCAGAACATCGGGAGAAAATGAACAGGATGCGGCAGCGCATTGCTCAGCGTCTGAAGGAGGCCCAGAATACATGTGCAATGCTGACAACTTTTAATGAGATTGACATGAGTAACATCCAGGAGATGAGGGCTCGGCACAAAGAGGCTTTTTTGAAGAAACATAACCTCAAACTAGGCTTCATGTCGGCATTTGTGAAGGCCTCAGCCTTTGCCTTGCAGGAACAGCCTGTTGTAAATGCAGTGATTGACGACACAACCAAAGAGGTGGTGTATAGGGATTATATTGACATCAGTGTTGCAGTGGCCACCCCACGGGGTCTGGTGGTTCCAGTCATCAGGA

In [1]:
EC_list=['5.1.3.3','5.3.1.9','5.4.2.2','4.1.2.13','2.7.1.165','3.1.1.31','5.1.3.1','2.2.1.1','1.1.1.42','1.2.4.2','1.5.1.3','2.3.1.61']
print("I searched for enzymes that had genes in all 3 organisms (some enzymes were only present in some of the 3 organisms)")

I searched for enzymes that had genes in all 3 organisms (some enzymes were only present in some of the 3 organisms)


In [54]:
for x in EC_list:
    x1="\'"+x+"\'"
    request_enz=REST.kegg_get("ec:" + x)
    open("ec_"+x+'.txt','w').write(request_enz.read())
    records_enz=Enzyme.parse(open("ec_"+x+'.txt'))
    record_enz=list(records_enz)[0]
    c.execute("""INSERT INTO pathway (name, enzymes) 
                VALUES (\'""" + record_enz.pathway[0][2]+"\', "+x1+');')
    for y in range(len(record_enz.genes)):
        if record_enz.genes[y][0]=='ECO':
            eco_gene=record_enz.genes[y][1][0]
            break
    c.execute("""INSERT INTO enzyme (name, EC, function, pathway, organism, gene)
                VALUES (\'"""+record_enz.name[0]+'\', '+x1+', \''+str(record_enz.reaction[0])+'\', \''+record_enz.pathway[0][2]+"\', 'Escherichia coli', \'"+eco_gene+'\');')
    Entrez.email='neilwang@berkeley.edu'
    handle=Entrez.esearch(db='nucleotide',term='Escherichia coli[ORGN] '+eco_gene,sort='relevance',idtype='acc')
    for i in Entrez.read(handle)['IdList']:
            handle=Entrez.efetch(db='nucleotide',id=i, rettype='fasta',retmode='text')
            break
    for record in SeqIO.parse(handle,'fasta'):
        eco_seq=record.seq
    c.execute("""INSERT INTO genes(name, description, nucleotide_seq, length)
                VALUES('ECO """+eco_gene+"', 'encodes enzyme "+x+" in E. Coli'"+', \''+str(eco_seq)+'\', '+str(len(eco_seq))+');')
              
              
    for y in range(len(record_enz.genes)):
        if record_enz.genes[y][0]=='HSA':
            hsa_gene=record_enz.genes[y][1][0]
            break
    c.execute("""INSERT INTO enzyme (name, EC, function, pathway, organism, gene)
                VALUES (\'"""+record_enz.name[0]+'\', '+x1+', \''+str(record_enz.reaction[0])+'\', \''+record_enz.pathway[0][2]+"\', 'Homo Sapiens', \'"+hsa_gene+'\');')

    Entrez.email='neilwang@berkeley.edu'
    handle=Entrez.esearch(db='nucleotide',term='Homo sapiens[ORGN] '+hsa_gene,sort='relevance',idtype='acc')
    for i in Entrez.read(handle)['IdList']:
            handle=Entrez.efetch(db='nucleotide',id=i, rettype='fasta',retmode='text')
            break
    for record in SeqIO.parse(handle,'fasta'):
        hsa_seq=record.seq
    c.execute("""INSERT INTO genes(name, description, nucleotide_seq, length)
                VALUES('HSA """+hsa_gene+"', 'encodes enzyme "+x+" in H. Sapiens'"+', \''+str(hsa_seq)+'\', '+str(len(hsa_seq))+');')
               
              
    for y in range(len(record_enz.genes)):
        if record_enz.genes[y][0]=='DME':
            dme_gene=record_enz.genes[y][1][0]
            break
    c.execute("""INSERT INTO enzyme (name, EC, function, pathway, organism, gene)
                VALUES (\'"""+record_enz.name[0]+'\', '+x1+', \''+str(record_enz.reaction[0])+'\', \''+record_enz.pathway[0][2]+"\', 'Drosophila melanogaster', \'"+dme_gene+'\');')

    Entrez.email='neilwang@berkeley.edu'
    handle=Entrez.esearch(db='nucleotide',term='Drosophila Melanogaster[ORGN] '+dme_gene,sort='relevance',idtype='acc')
    for i in Entrez.read(handle)['IdList']:
            handle=Entrez.efetch(db='nucleotide',id=i, rettype='fasta',retmode='text')
            break
    for record in SeqIO.parse(handle,'fasta'):
        dme_seq=record.seq
    c.execute("""INSERT INTO genes(name, description, nucleotide_seq, length)
                VALUES('DME """+dme_gene+"', 'encodes enzyme "+x+" in D. Melanogaster'"+', \''+str(dme_seq)+'\', '+str(len(dme_seq))+');')
conn.commit()